In [1]:
import requests
from pyquery import PyQuery as pq
import re
import json

In [2]:
#Making list of teams
## Keep for local testing
bpl2012 = requests.get("https://en.wikipedia.org/wiki/2012–13_Premier_League").text
team_table = pq(bpl2012).find('.wikitable')[1]
team_list_tab = pq(team_table)('tr')
team_list = []
for i in range(1,len(team_list_tab)):
    row = pq(team_list_tab[i])('td')[0].text
    team_list.append(row)

In [245]:
#Getting transfers for the season 2012 summer
transfers_sum_2012 = requests.get('https://en.wikipedia.org/wiki/List_of_English_football_transfers_summer_2013').text

In [ ]:
## Keep for local testing
transfer_table = pq(transfers_sum_2012).find('.wikitable')[0]
transfer_list_tab = pq(transfer_table)('tr')
tr_list = []
for i in range(1,len(transfer_list_tab)):
    transfer = {}
    td = pq(transfer_list_tab[i])('td')
    date = pq(td[0])('span')[1].text
    try:
        name = pq(td[1])('span')[1].text
    except:
        name = pq(td[1])('a')[0].text
    try:
        from_club = pq(td[2])('a')[-1].text
    except:
        continue
    
    try:
        to_club = pq(td[3])('a')[-1].text
    except:
        continue
    if name == None:
        try:
            name = pq(td[1])('a')[0].text
        except:
            name = pq(td[1])('span')[0].text
    if name == None:
        name = pq(td[1])('a')[1].text
    if pq(td[4])('td')[0].text != None:
        value = pq(td[4])('td')[0].text
    else:
        try:
            t_value = pq(td[4])('span')[0].text
            t_value = re.findall(r'\d+',t_value)[0]
            p_value = t_value[4:]
            value_code = t_value[0:4]
            ref_value_code = 7007
            div = 10000000
            if ref_value_code - int(value_code) > 0:
                div = div*(10**(ref_value_code - int(value_code)))
            value = int(p_value)/div
        except:
            value = 'No Value'
    if value == 0:
        value = 'Free'

    print i,date,name,from_club,to_club,value
    if from_club in team_list or to_club in team_list:
        transfer['date'] = date
        transfer['name'] = name
        transfer['from'] = from_club
        transfer['to'] = to_club
        transfer['value'] = value
        tr_list.append(transfer)

In [3]:
def gen_transfers(league_url,transfers_url,league_code,currency):
    bpl2012 = requests.get(league_url).text
    team_table = pq(bpl2012).find('.wikitable')[0]
    team_list_tab = pq(team_table)('tr')
    team_list = []
    for i in range(1,len(team_list_tab)):
        if league_code == 1:
            row = pq(pq(pq(team_list_tab[i])('td'))('a')[0]).attr.title
        else:
            row = pq(pq(team_list_tab[i])('td'))('a')[0].text
        team_list.append(row)
    transfers_data = requests.get(transfers_url).text
    transfer_table = pq(transfers_data).find('.wikitable')[0]
    transfer_list_tab = pq(transfer_table)('tr')
    tr_list = []
    for i in range(1,len(transfer_list_tab)):
        transfer = {}
        td = pq(transfer_list_tab[i])('td')
        date = pq(td[0])('span')[1].text
        try:
            name = pq(td[1])('span')[1].text
        except:
            name = pq(td[1])('a')[0].text
        try:
            from_club = pq(td[2])('a')[-1].text
        except:
            continue
        try:
            to_club = pq(td[3])('a')[-1].text
        except:
            continue
        if name == None:
            try:
                name = pq(td[1])('a')[0].text
            except:
                name = pq(td[1])('span')[0].text
        if name == None:
            name = pq(td[1])('a')[1].text
        #Code for getting values
        #league_code is 1 for troublesome pages where sortkey does not have a value
        if league_code == 1:
            t_value = pq(td[4])('td')[0].text
            if t_value == None:
                value = pq(td[4]).text().split(' ')[1]
            else:
                try:
                    num_value = re.findall(r"\d+.\d+",t_value)[0]
                except:
                    num_value = re.findall(r"\d+",t_value)[0]
                try:
                    mul = re.findall(r"[a-zA-Z]",t_value)[0]
                    if mul == 'm':
                        mul_val = 10**6
                    else:
                        mul_val = 10**3
                    value = float(num_value)*mul_val
                except:
                    continue
                    
        else:
            if pq(td[4])('td')[0].text != None:
                value = pq(td[4])('td')[0].text
            else:
                try:
                    t_value = pq(td[4])('span')[0].text
                    t_value = re.findall(r'\d+',t_value)[0]
                    p_value = t_value[4:]
                    value_code = t_value[0:4]
                    ref_value_code_1 = 7007
                    ref_value_code_2 = 7001
                    if ref_value_code_1 - int(value_code) > 4:
                        ref_value_code = ref_value_code_2
                    else:
                        ref_value_code = ref_value_code_1
                    div = 10000000
                    if ref_value_code - int(value_code) > 0:
                        div = div*(10**(ref_value_code - int(value_code)))
                    value = int(p_value)/div
                except:
                    value = 'No Value'
            if value == 0:
                try:
                    if pq(td[4]).text().split(' ')[2] == 'return':
                        value = 'Loan return'
                    else:
                        value = pq(td[4]).text().split(' ')[1]
                except:
                    value = 'Free'

        print i,date,name,from_club,to_club,value
        if from_club in team_list or to_club in team_list:
            transfer['date'] = date
            transfer['name'] = name
            transfer['from'] = from_club
            transfer['to'] = to_club
            transfer['value'] = value
            transfer['currency'] = currency
            tr_list.append(transfer)
    return tr_list

In [464]:
#Creating dictionaries for Spain and English leagues
list_to_be_scraped = {}
list_to_be_scraped['Spain_winter_2012_13'] = {'league_url':'https://en.wikipedia.org/wiki/2012%E2%80%9313_La_Liga','transfer_url':'https://en.wikipedia.org/wiki/List_of_Spanish_football_transfers_winter_2012–13','league_code':1,'currency':'Euro'}
list_to_be_scraped['Spain_summer_2013'] = {'league_url':'https://en.wikipedia.org/wiki/2013%E2%80%9314_La_Liga','transfer_url':'https://en.wikipedia.org/wiki/List_of_Spanish_football_transfers_summer_2013','league_code':0,'currency':'Euro'}
list_to_be_scraped['Spain_winter_2013_14'] = {'league_url':'https://en.wikipedia.org/wiki/2013%E2%80%9314_La_Liga','transfer_url':'https://en.wikipedia.org/wiki/List_of_Spanish_football_transfers_winter_2013–14','league_code':0,'currency':'Euro'}
list_to_be_scraped['England_winter_2012_13'] = {'league_url':'https://en.wikipedia.org/wiki/2012–13_Premier_League','transfer_url':'https://en.wikipedia.org/wiki/List_of_English_football_transfers_winter_2012–13','league_code':0,'currency':'Pound'}
list_to_be_scraped['England_winter_2013_14'] = {'league_url':'https://en.wikipedia.org/wiki/2013–14_Premier_League','transfer_url':'https://en.wikipedia.org/wiki/List_of_English_football_transfers_winter_2013–14','league_code':0,'currency':'Pound'}
list_to_be_scraped['England_summer_2013'] = {'league_url':'https://en.wikipedia.org/wiki/2013–14_Premier_League','transfer_url':'https://en.wikipedia.org/wiki/List_of_English_football_transfers_summer_2013','league_code':0,'currency':'Pound'}
list_to_be_scraped['England_summer_2014'] = {'league_url':'https://en.wikipedia.org/wiki/2014–15_Premier_League','transfer_url':'https://en.wikipedia.org/wiki/List_of_English_football_transfers_summer_2014','league_code':0,'currency':'Pound'}

In [466]:
#Scraping and storing in the Transfer_data folder
for key in list_to_be_scraped.keys():
    league_url = list_to_be_scraped[key]['league_url']
    transfer_url = list_to_be_scraped[key]['transfer_url']
    league_code = list_to_be_scraped[key]['league_code']
    currency = list_to_be_scraped[key]['currency']
    tr_list = gen_transfers(league_url,transfer_url,league_code,currency)
    with open('Transfer_data/'+key+'.json', 'w') as fp:
        json.dump(tr_list, fp)

###Code for scraping the Italian Leagues

In [80]:
def gen_transfers_italy(league_url,transfers_url,league_code,currency):
    bpl2012 = requests.get(league_url).text
    team_table = pq(bpl2012).find('.wikitable')[1]
    team_list_tab = pq(team_table)('tr')
    team_list = []
    for i in range(1,len(team_list_tab)):
        row = pq(pq(team_list_tab[i])('td'))('a')[0].text
        team_list.append(row)
    #print team_list
    transfers_data = requests.get(transfers_url).text
    transfer_table = pq(transfers_data).find('.wikitable')[0]
    transfer_list_tab = pq(transfer_table)('tr')
    tr_list = []
    for i in range(1,len(transfer_list_tab)):
        transfer = {}
        td = pq(transfer_list_tab[i])('td')
        date = pq(td[0])('span')[1].text
        try:
            name = pq(td[1])('span')[1].text
        except:
            name = pq(td[1])('a')[0].text
        try:
            from_club = pq(td[2])('a')[-1].text
        except:
            continue
        if from_club == None:
            from_club = pq(td[2])('a')[0].text
        try:
            to_club = pq(td[3])('a')[-1].text
        except:
            continue
        if to_club == None:
            to_club = pq(td[3])('a')[0].text
        if name == None:
            try:
                name = pq(td[1])('a')[0].text
            except:
                name = pq(td[1])('span')[0].text
        if name == None:
            name = pq(td[1])('a')[1].text
        #Code for getting values
        #league_code is 1 for troublesome pages where sortkey does not have a value
        if league_code == 1:
            t_value = pq(td[4])('td')[0].text
            if t_value == None:
                value = pq(td[4]).text().split(' ')[1]
            else:
                try:
                    num_value = re.findall(r"\d+.\d+",t_value)[0]
                except:
                    value = pq(td[4])('td')[0].text
                try:
                    num_value = re.findall(r"\d+",t_value)[0]
                except:
                    value = pq(td[4])('td')[0].text
                try:
                    mul = re.findall(r"[a-zA-Z]",t_value)[0]
                    if mul == 'M' or mul == 'm':
                        mul_val = 10**6
                    else:
                        mul_val = 10**3
                    value = float(num_value)*mul_val
                except:
                    value = pq(td[4])('td')[0].text
            if value < 50000:
                value = pq(td[4])('td')[0].text
                    
        else:
            if pq(td[4])('td')[0].text != None:
                value = pq(td[4])('td')[0].text
            else:
                try:
                    t_value = pq(td[4])('span')[0].text
                    t_value = re.findall(r'\d+',t_value)[0]
                    p_value = t_value[4:]
                    value_code = t_value[0:4]
                    ref_value_code = 7004
                    div = 10000000
                    if ref_value_code - int(value_code) > 0:
                        div = div*(10**(ref_value_code - int(value_code)))
                    value = int(p_value)/div
                except:
                    value = 'No Value'
            if value == 0:
                try:
                    if pq(td[4]).text().split(' ')[2] == 'return':
                        value = 'Loan return'
                    else:
                        value = pq(td[4]).text().split(' ')[1]
                except:
                    value = 'Free'

        #print i,date,name,from_club,to_club,value
        if from_club in team_list or to_club in team_list:
            transfer['date'] = date
            transfer['name'] = name
            transfer['from'] = from_club
            transfer['to'] = to_club
            transfer['value'] = value
            transfer['currency'] = currency
            tr_list.append(transfer)
    return tr_list

In [82]:
list_to_be_scraped_it = {}
list_to_be_scraped_it['Italy_summer_2013_Aug'] = {'league_url':'https://en.wikipedia.org/wiki/2013–14_Serie_A','transfer_url':'https://en.wikipedia.org/wiki/List_of_Italian_football_transfers_summer_2013_(August)','league_code':1,'currency':'Euro'}
list_to_be_scraped_it['Italy_summer_2013_Jul'] = {'league_url':'https://en.wikipedia.org/wiki/2013–14_Serie_A','transfer_url':'https://en.wikipedia.org/wiki/List_of_Italian_football_transfers_summer_2013_(July)','league_code':0,'currency':'Euro'}
list_to_be_scraped_it['Italy_winter_2013_14'] = {'league_url':'https://en.wikipedia.org/wiki/2013–14_Serie_A','transfer_url':'https://en.wikipedia.org/wiki/List_of_Italian_football_transfers_winter_2013–14','league_code':1,'currency':'Euro'}
list_to_be_scraped_it['Italy_winter_2012_13'] = {'league_url':'https://en.wikipedia.org/wiki/2012–13_Serie_A','transfer_url':'https://en.wikipedia.org/wiki/List_of_Italian_football_transfers_winter_2012–13','league_code':0,'currency':'Euro'}
list_to_be_scraped_it['Italy_summer_2014'] = {'league_url':'https://en.wikipedia.org/wiki/2014–15_Serie_A','transfer_url':'https://en.wikipedia.org/wiki/List_of_Italian_football_transfers_summer_2014','league_code':1,'currency':'Euro'}

In [83]:
for key in list_to_be_scraped_it.keys():
    league_url = list_to_be_scraped_it[key]['league_url']
    transfer_url = list_to_be_scraped_it[key]['transfer_url']
    league_code = list_to_be_scraped_it[key]['league_code']
    currency = list_to_be_scraped_it[key]['currency']
    tr_list = gen_transfers_italy(league_url,transfer_url,league_code,currency)
    with open('Transfer_data/'+key+'.json', 'w') as fp:
        json.dump(tr_list, fp)